# LangChain Basic

1. 패키지 설치
2. 기본 패키지 설정
3. Azure 환경 설정
4. LLM
5. Embedding
6. Prompt Template
7. Chain

In [1]:
%%writefile requirements.txt
openai
langchain
tiktoken
unstructured

Overwriting requirements.txt


In [2]:
!pip install -r requirements.txt

In [3]:
!pip install openai==0.28.0

In [4]:
# 2. 기본 패키지 설정

import openai
import langchain

print(openai.__version__)
print(langchain.__version__)


0.28.0
0.0.334


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# 3. OpenAI 환경설정

import os

api_key_file_path = '/content/drive/MyDrive/Colab Notebooks/api_key.txt'

# Google Drive에서 API 키를 읽어옴
with open(api_key_file_path, 'r') as file:
    openai_api_key = file.read().strip()

# OpenAI API 키를 환경 변수로 설정
os.environ["OPENAI_API_KEY"] = openai_api_key

openai.api_key = os.getenv("OPENAI_API_KEY")
print(openai.api_type)
print(openai.__version__)

open_ai
0.28.0


In [ ]:
# 4. OpenAI LLM(CompletionModel 연결)
from langchain.llms import OpenAI
com_llm = OpenAI(model_name="gpt-3.5-turbo-instruct")
print(com_llm("내 건성 화장품 추천해줘"))



1. 에뛰드하우스 모이스트풀 씨드 히어로 씨드 블램
2. 라네즈 레이어링 커호뉴 3번 세럼
3. 더페이스샵 씨드 앤 씨드 씨드 트리트먼트
4. 스킨푸드 쑥 젤리 수분크림
5. 이니스프리 그린티 씨드 크림
6. 바이오더마 아토덤 크림
7. 코스알엑스 모이스처라이징 크림
8. 닥터자르트 세라마이드 크림
9. 라비오뜨 레베르 에센셜 모이스춰라이저
10. 미샤 더 퍼스트 트


In [ ]:
# 4-1. OpenAI LLM(Chat Model 연결)

from langchain.chat_models import ChatOpenAI

chat_llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0
)

In [ ]:
# 4-2. Chat llm 사용

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

messages = [
    SystemMessage(content="한글을 이태리어로 번역해주고, 한글 발음을 함께 알려주는 보조."),
    HumanMessage(content="피자 정말 맛있네요."),
    AIMessage(content="Questo pizza è davvero delizioso(쿼스토 피짜 에 데리보 델리치오소)"),
    HumanMessage(content="아이스크림 두개 주세요."), # 유저의 질문
]


In [ ]:
chat_llm(messages).content

'Due gelati per favore(두에 젤라티 페르 파보레)'

In [ ]:
# 5-1. Embedding 모델 연결

from langchain.embeddings.openai import OpenAIEmbeddings

# 임베딩 모델 생성
embeddings = OpenAIEmbeddings(
    model='text-embedding-ada-002')

embeddings.embed_query("이 와인 얼마인가요?")

[0.01845273096892258,
 -0.02809905678961399,
 0.006133942557836497,
 -0.02425579791562262,
 -0.02909168641699096,
 -0.004317302972067709,
 -0.036141900399455945,
 0.001348162933764857,
 -0.02358131737703694,
 -0.0009353626051300338,
 -0.001590752661706063,
 0.010887111923590093,
 -0.02599926162772111,
 -0.01614932076273229,
 0.017281937479293474,
 0.039450665824045855,
 0.0172437588435805,
 -0.0034169369064030964,
 0.0017593724471065345,
 -0.00694840785730007,
 -0.025782919129756,
 -0.014876719750954472,
 -0.019788964592426243,
 -0.0007452676281793023,
 -0.001638475257855389,
 0.024064906599702784,
 -0.0006744791416018861,
 -0.007342914683178584,
 0.013451405127647273,
 -0.016531101531926898,
 0.010428974814292058,
 0.0013934993651821223,
 -0.012255158593327826,
 0.015169417657700485,
 -0.002911077433786366,
 -0.008259187970452128,
 0.006795695642754479,
 -0.0009234319560927022,
 0.009423618664569916,
 0.006398007419120305,
 0.011135269330434336,
 0.0038687105216979167,
 0.016874705528

In [ ]:
# 5-2. 유사도

from numpy import dot
from numpy.linalg import norm

def cos_sim(word1, word2):
    em1 = embeddings.embed_query(word1)
    em2 = embeddings.embed_query(word2)
    return dot(em1, em2)/(norm(em1)*norm(em2))

In [ ]:
cos_sim("고양이", "강아지") # 완전히 동일하면 1, 다를수록 0

0.9074521052331773

In [ ]:
# 6. Prompt Template(Zero-shot-learning)

from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

system_template = """
한글 텍스트가 제공됩니다.
당신의 임무는 사용자의 피부타입에 맞는 화장품을 추천하는 것입니다.
화장품 브랜드와 제품명, 간단한 특징을 제공합니다.
"""

system_message_prompt_template = SystemMessagePromptTemplate.from_template(
    system_template)

human_template = "{input_text}"

human_message_prompt_template = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_template = ChatPromptTemplate.from_messages( # 최종 템플릿
    [system_message_prompt_template, human_message_prompt_template])


In [ ]:
final_prompt = chat_prompt_template.format_prompt(
                          input_text="DRPT(Dry, Resistant, Pigmented, Tight)").to_messages() # 변수값만 전달해 최종 프롬프트 완성
print(final_prompt)

[SystemMessage(content='\n한글 텍스트가 제공됩니다.\n사용자의 피부타입에 맞는 화장품을 추천하는 것입니다.\n화장품 브랜드와 제품명, 간단한 특징을 제공합니다.\n'), HumanMessage(content='DRPT(Dry, Resistant, Pigmented, Tight)')]


In [ ]:
chat_llm(final_prompt)

AIMessage(content='DRPT 피부타입에 맞는 화장품 추천입니다.\n\n1. 브랜드: Clinique\n   제품명: Moisture Surge 72-Hour Auto-Replenishing Hydrator\n   특징: 촉촉한 수분 공급과 피부 장벽 강화를 도와주는 제품입니다. 피부를 촉촉하게 유지하며 건조함을 완화시켜줍니다.\n\n2. 브랜드: La Roche-Posay\n   제품명: Toleriane Double Repair Face Moisturizer\n   특징: 피부 장벽을 강화하고 수분을 공급하여 건조함을 완화시켜주는 제품입니다. 피부를 부드럽고 탄력있게 가꿔줍니다.\n\n3. 브랜드: Cetaphil\n   제품명: Daily Facial Moisturizer with SPF 15\n   특징: 촉촉한 수분 공급과 함께 자외선 차단 기능을 가진 제품입니다. 피부를 보호하고 건조함을 예방해줍니다.\n\n4. 브랜드: Neutrogena\n   제품명: Hydro Boost Water Gel\n   특징: 수분을 공급하고 피부를 촉촉하게 유지시켜주는 제품입니다. 가벼운 젤 텍스처로 피부에 빠르게 흡수되어 건조함을 완화시켜줍니다.\n\n5. 브랜드: Aveeno\n   제품명: Positively Radiant Daily Moisturizer\n   특징: 수분을 공급하고 피부를 밝고 환하게 가꿔주는 제품입니다. 피부를 촉촉하게 유지하며 건조함을 예방해줍니다.\n\n위의 제품들은 DRPT 피부타입에 맞는 화장품으로, 각각의 브랜드에서 다양한 제품을 제공하고 있습니다. 피부타입과 선호하는 브랜드에 따라 적합한 제품을 선택해보세요.')

In [ ]:
from langchain.prompts import FewShotChatMessagePromptTemplate

# fewshot 예시 문장
examples = [
    {"input": "DRPT(Dry, Resistant, Pigmented, Tight)", "output": "고운 피부결을 갖고 계시지만 자외선은 꼭 피해야 하는 피부를 갖고 계시네요. SPF 30이상의 선크림을 추천드립니다. SPF 50+이고 마무리감이 촉촉한 라네즈의 래디언씨 선크림입니다. 잡티, 색소침착에도 효과적이에요."},
    {"input": "DRPW(Dry, Resistant, Pigmented, Wrinkled)", "output": "자극에 강하지만 건조하고 주름이 많은 피부를 갖고 계시네요. 레티놀은 피부가 튼튼하고 단단해지게 해줘서 주름개선에 효과가 좋습니다. 처음부터 높은 용량을 쓰면 부작용이 있을 수 있으니 레티놀 0.2% 함량의 디오디어리 스쿠알란 세럼을 추천드려요. 자외선 차단제도 잊지 마세요!"},
    {"input": "DRNT(Dry, Resistant, Non-Pigmented, Tight)", "output": "피부고민 없는 건강한 피부를 갖고 계시네요. 다만 피부가 건조한 편이시라면 수분 관리가 필요합니다. 키엘의 울트라 페이셜 수분크림을 추천해드려요. 주기적인 각질관리도 잊지 마세요!"},
    {"input": "DRNW(Dry, Resistant, Non-Pigmented, Wrinkled)", "output": "미세한 잔주름이 있는 피부신가요? 비타민은 피부 노화 방지에 효과적인데요, 피부 표면에서 이물질들이 산화되는 것을 막아줍니다. 비타민 E가 들어간 시드물의 민중기 앰플 추천드려요."},
    {"input": "DSPT(Dry, Sensitive, Pigmented, Tight)", "output": "항상 피부 문제로 고민이 많으셨죠? 보습을 1순위로, 순한 제품만 쓰셔야 합니다. 닥터지의 레드 블레미쉬 클리어 수딩 크림 추천드려요. 그리고 어떤 화장품이든 사용하시기 전에 패치테스트 잊지 마세요!"},
    {"input": "DSPW(Dry, Sensitive, Pigmented, Wrinkled)", "output": "민감하고 건조하고 색소, 주름까지.. 피부 걱정 많으셨겠어요. 순한 민감성 제품만 쓰셔야 합니다. 닥터지의 레드 블레미쉬 클리어 수딩 크림 추천드려요. 그리고 어떤 화장품이든 사용하시기 전에 패치테스트 잊지 마세요!"},
    {"input": "DSNT(Dry, Sensitive, Non-Pigmented, Tight)", "output": "극심한 건조함과 화끈거림 때문에 고생중이신가요? 저자극 약산성 클렌징 제품을 사용해보세요. 염증반응을 억제하는 오메가3 섭취도 필요합니다. 제품은 비플레인 녹두 약산성 클렌징폼을 추천드려요."},
    {"input": "DSNW(Dry, Sensitive, Non-Pigmented, Wrinkled)", "output": "균일한 피부톤과 주름을 갖고 계시네요. 속건조가 있으신가요? 염증반응을 억제하는 오메가3를 드셔보세요. 피부장벽 튼튼하게 해주는 위치하젤 성분이 필요합니다. 제품은 AHC의 허브 솔루션 토너 추천드려요!"},
    {"input": "ORPT(Oily, Resistant, Pigmented, Tight)", "output": "얼굴에 유분기가 많고 어두운 피부톤이지만 주름과 자극에 강한 피부를 갖고 계시네요. SPF 30이상의 선크림과 필링 제품 사용을 추천드립니다. SPF 50+인 벤튼의 시카 수분 선세럼입니다."},
    {"input": "ORPW(Oily, Resistant, Pigmented, Wrinkled)", "output": "모공이 넓지만 여드름은 쉽게 나지 않는 피부를 갖고 계시네요. SPF 30이상의 선크림과 항산화제 사용을 추천드립니다. 비타민은 피부 노화 방지에 효과적인데요, 피부 표면에서 이물질들이 산화되는 것을 막아줍니다. 비타민 E가 들어간 시드물의 민중기 앰플 추천드려요."},
    {"input": "ORNT(Oily, Resistant, Non-Pigmented, Tight)", "output": "흠잡을 것 하나 없는 건강한 탱탱볼 피부를 갖고 계시네요. 지성 피부를 갖고 계셔서 피지 흡착에 좋은 AHA 성분 추천드려요. 이니스프리의 수퍼 화산송이 모공 마스크입니다."},
    {"input": "ORNW(Oily, Resistant, Non-Pigmented, Wrinkled)", "output": "여드름은 없지만 잔주름이 신경쓰이는 지성피부시네요. 주름을 케어해주기로 유명한 콜라겐 화장품 추천드립니다. 네이처카인드의 콜라겐 크림입니다."},
    {"input": "OSPT(Oily, Sensitive, Pigmented, Tight)", "output": "지성이지만 민감성 피부라 피지, 염증, 색소침착 때문에 많이 신경쓰이시죠? 필링은 자제하시고 민감성 제품만 사용하세요. 피부 장벽 강화에 좋은 위치하젤 성분이 필요합니다. 제품은 AHC의 허브 솔루션 토너 추천드려요!"},
    {"input": "OSPW(Oily, Sensitive, Pigmented, Wrinkled)", "output": "지성, 민감성 피부에 색소와 주름도 갖고 계시네요. 반드시 민감성 제품만 사용하시고, 패치테스트는 필수에요. 레티놀은 피부가 튼튼하고 단단해지게 해줘서 주름개선에 효과가 좋습니다. 처음부터 높은 용량을 쓰면 부작용이 있을 수 있으니 레티놀 0.2% 함량의 디오디어리 스쿠알란 세럼을 추천드려요."},
    {"input": "OSNT(Oily, Sensitive, Non-Pigmented, Tight)", "output": "여드름이 올라오지만 자국이 남지는 않는 피부를 갖고 계시네요. 염증반응을 억제하는 오메가3를 드셔보세요. 피부장벽 튼튼하게 해주는 위치하젤 성분이 필요합니다. 제품은 AHC의 허브 솔루션 토너 추천드려요!"},
    {"input": "OSNW(Oily, Sensitive, Non-Pigmented, Wrinkled)", "output": "햇빛에 쉽게 붉어지고 번들거리는 피부를 갖고 계시네요. SPF 30이상의 선크림과 항산화제 사용을 추천드립니다. 비타민은 피부 노화 방지에 효과적인데요, 피부 표면에서 이물질들이 산화되는 것을 막아줍니다. 비타민 E가 들어간 시드물의 민중기 앰플 추천드려요."}
]

# fewshot 예시 문장 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"), # 사람이 질문
        ("ai", "{output}"), # ai가 답변
    ]
)

# fewshot 예시 문장+템플릿 결합
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: DRPT(Dry, Resistant, Pigmented, Tight)
AI: 고운 피부결을 갖고 계시지만 자외선은 꼭 피해야 하는 피부를 갖고 계시네요. SPF 30이상의 선크림을 추천드립니다. SPF 50+이고 마무리감이 촉촉한 라네즈의 래디언씨 선크림입니다. 잡티, 색소침착에도 효과적이에요.
Human: DRPW(Dry, Resistant, Pigmented, Wrinkled)
AI: 자극에 강하지만 건조하고 주름이 많은 피부를 갖고 계시네요. 레티놀은 피부가 튼튼하고 단단해지게 해줘서 주름개선에 효과가 좋습니다. 처음부터 높은 용량을 쓰면 부작용이 있을 수 있으니 레티놀 0.2% 함량의 디오디어리 스쿠알란 세럼을 추천드려요. 자외선 차단제도 잊지 마세요!
Human: DRNT(Dry, Resistant, Non-Pigmented, Tight)
AI: 피부고민 없는 건강한 피부를 갖고 계시네요. 다만 피부가 건조한 편이시라면 수분 관리가 필요합니다. 키엘의 울트라 페이셜 수분크림을 추천해드려요. 주기적인 각질관리도 잊지 마세요!
Human: DRNW(Dry, Resistant, Non-Pigmented, Wrinkled)
AI: 미세한 잔주름이 있는 피부신가요? 비타민은 피부 노화 방지에 효과적인데요, 피부 표면에서 이물질들이 산화되는 것을 막아줍니다. 비타민 E가 들어간 시드물의 민중기 앰플 추천드려요.
Human: DSPT(Dry, Sensitive, Pigmented, Tight)
AI: 항상 피부 문제로 고민이 많으셨죠? 보습을 1순위로, 순한 제품만 쓰셔야 합니다. 닥터지의 레드 블레미쉬 클리어 수딩 크림 추천드려요. 그리고 어떤 화장품이든 사용하시기 전에 패치테스트 잊지 마세요!
Human: DSPW(Dry, Sensitive, Pigmented, Wrinkled)
AI: 민감하고 건조하고 색소, 주름까지.. 피부 걱정 많으셨겠어요. 순한 민감성 제품만 쓰셔야 합니다. 닥터지의 레드 블레미쉬 클리어 수딩 크림 추천드려요. 그리고 어떤

In [ ]:
# ChatPromptTemplate

system_template = """
한글 텍스트가 제공됩니다.
당신의 임무는 사용자의 피부타입에 맞는 화장품을 추천하는 것입니다.
화장품 브랜드와 제품명, 간단한 특징을 제공합니다.
"""

system_message_prompt_template = SystemMessagePromptTemplate.from_template(
    system_template)

human_template = "{input}"

human_message_prompt_template = HumanMessagePromptTemplate.from_template(human_template)

fewshot_chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt_template, few_shot_prompt, human_message_prompt_template])

In [ ]:
final_prompt = fewshot_chat_prompt_template.format_prompt(
    input = "OSPT(Oily, Sensitive, Pigmented, Tight)"
).to_messages()
chat_llm(final_prompt).content


'지성이지만 민감성 피부이며 색소침착과 탄력이 걱정되시는군요. 자외선 차단과 색소침착 개선에 효과적인 선크림을 추천드립니다. SPF 50+이고 마무리감이 촉촉한 라네즈의 래디언씨 선크림입니다. 잡티, 색소침착에도 효과적이에요.'

In [ ]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
  chat_llm(final_prompt).content
  print(cb) # 콜백 함수 결과 출력(토큰수, 과금)

Tokens Used: 718
	Prompt Tokens: 674
	Completion Tokens: 44
Successful Requests: 1
Total Cost (USD): $0.0010990000000000002


In [ ]:
# 7-1. Chain 연결

from langchain.chains import LLMChain

system_template = """
한글 텍스트가 제공됩니다.
당신의 임무는 사용자의 피부타입에 맞는 화장품을 추천하는 것입니다.
화장품 브랜드와 제품명, 간단한 특징을 제공합니다.
"""

system_message_prompt_template = SystemMessagePromptTemplate.from_template(
    system_template)

human_template = "{input_text}"

human_message_prompt_template = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt_template, human_message_prompt_template])



In [ ]:
chain = LLMChain(llm=chat_llm, prompt=fewshot_chat_prompt_template)
chain.run(input="DRPT")

'DRPT(Dry, Resistant, Pigmented, Tight)에 맞는 화장품을 추천해드리겠습니다. 건조한 피부에는 촉촉한 보습 제품이 필요하며, 색소침착에도 효과적인 제품이 좋습니다. 또한 피부 탄력을 유지하기 위해 탄력 개선 제품도 고려해야 합니다.\n\n이러한 요구사항을 충족하는 제품으로는 라네즈의 래디언씨 선크림을 추천드립니다. 이 제품은 SPF 50+로 자외선 차단 효과가 뛰어나며, 촉촉한 마무리감을 제공하여 건조한 피부에 적합합니다. 또한 잡티와 색소침착에도 효과적이어서 피부 톤을 밝게 유지할 수 있습니다.'

In [ ]:
#7-2. Chain-Prompt 연결

from langchain import PromptTemplate

template = """"
한글 텍스트 {input_text}가 제공됩니다.
당신의 임무는 사용자의 피부타입에 맞는 화장품을 추천하는 것입니다.
화장품 브랜드와 제품명, 간단한 특징을 제공합니다.
"""

cosmetic_prompt = PromptTemplate(
    input_variables=['input_text'],
    template=template
)

chain = LLMChain(llm=chat_llm,
                 prompt=cosmetic_prompt)

In [ ]:
chain.run(input_text="DSRW")

'사용자의 피부타입에 따라 추천하는 화장품은 다음과 같습니다:\n\n1. 브랜드: Innisfree\n   제품명: 그린티 씨드 크림\n   특징: 보습력이 뛰어나고 가벼운 텍스처로 피부에 촉촉함을 유지해주는 제품입니다. 녹차 추출물이 함유되어 피부를 진정시켜주고 피부 톤을 밝게 만들어줍니다.\n\n2. 브랜드: Laneige\n   제품명: 워터 슬리핑 마스크\n   특징: 수분 부족으로 인한 건조함을 개선해주는 수면 마스크입니다. 수분을 공급하여 피부를 촉촉하게 유지해주고, 잠들어 있는 동안 피부를 진정시켜줍니다.\n\n3. 브랜드: Etude House\n   제품명: 플레이 컬러 아이즈\n   특징: 다양한 컬러와 텍스처로 구성된 아이섀도우 팔레트입니다. 오래 지속되는 지속력과 발색력이 있어 메이크업을 오랫동안 유지할 수 있습니다.\n\n4. 브랜드: Missha\n   제품명: M 퍼펙트 커버 BB 크림\n   특징: 커버력이 뛰어나고 자연스러운 마무리감을 제공하는 BB 크림입니다. 자외선 차단 기능이 있어 피부를 보호하고, 촉촉한 텍스처로 피부에 촉촉함을 유지해줍니다.\n\n5. 브랜드: Sulwhasoo\n   제품명: 기초 수분 에센스\n   특징: 피부에 수분을 공급하여 건조함을 개선해주는 에센스입니다. 한방 재료를 사용하여 피부를 진정시켜주고, 피부 탄력을 증가시켜줍니다.\n\n이러한 제품들은 각각의 피부타입에 맞는 화장품으로, 사용자의 피부타입에 따라 적합한 제품을 선택할 수 있습니다.'

In [ ]:
# 7-3 두개 체인 연결
# 하나의 input variable 만 사용가능
from langchain.chains import SimpleSequentialChain # 간단하게 체인을 연결

# 첫번째 체인(화장품 추천)
chain = LLMChain(llm = chat_llm,
                 prompt = cosmetic_prompt)

# 두번째 체인(스킨케어 방법)
second_prompt = PromptTemplate(
    input_variables=["user input"],
    template="다음 화장품을 이용한 스킨케어 방법 {skin care}",
)

chain_two = LLMChain(llm=chat_llm, prompt=second_prompt)

# 두가지 체인 결합
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

In [ ]:
overall_chain.run("DRPW")



> Entering new SimpleSequentialChain chain...
사용자의 피부타입에 따라 추천하는 화장품은 다음과 같습니다:

1. 브랜드: Innisfree
   제품명: 그린티 씨드 크림
   특징: 보습력이 뛰어나고 가벼운 텍스처로 피부에 촉촉함을 유지해주는 제품입니다. 녹차 추출물이 함유되어 피부를 진정시켜주고 피부 톤을 밝게 만들어줍니다.

2. 브랜드: Laneige
   제품명: 워터 슬리핑 마스크
   특징: 수분 부족으로 인한 건조함을 개선해주는 수면 마스크입니다. 수분을 공급하여 피부를 촉촉하게 유지해주고, 잠들어 있는 동안 피부를 진정시켜줍니다.

3. 브랜드: Etude House
   제품명: 플레이 컬러 아이즈
   특징: 다양한 컬러와 텍스처로 구성된 아이섀도우 팔레트입니다. 부드러운 발림성과 오래 지속되는 지속력을 가지고 있어 메이크업을 오랫동안 유지시켜줍니다.

4. 브랜드: Missha
   제품명: M 퍼펙트 커버 BB 크림
   특징: 커버력이 뛰어나고 자연스러운 마무리감을 가진 BB 크림입니다. 자외선 차단 기능이 있어 피부를 보호해주고, 촉촉한 텍스처로 피부에 촉촉함을 유지시켜줍니다.

5. 브랜드: Sulwhasoo
   제품명: 기초 수분 에센스
   특징: 피부에 수분을 공급하여 건조함을 개선해주는 에센스입니다. 한방 재료를 사용하여 피부를 진정시켜주고, 피부 탄력을 증진시켜줍니다.

이러한 제품들은 각각 다른 피부타입에 맞게 개발되었으며, 사용자의 피부타입에 따라 적합한 제품을 선택하여 추천해줄 수 있습니다.
피부타입에 따라 추천하는 화장품은 다음과 같습니다:

1. 건성 피부: Innisfree의 그린티 씨드 크림은 보습력이 뛰어나고 촉촉한 텍스처로 피부에 영양과 수분을 공급해줍니다. 녹차 추출물이 함유되어 피부를 진정시켜주고 피부 톤을 밝게 만들어줍니다.

2. 수분 부족한 피부: Laneige의 워터 슬리핑 마스크는 수분을 공급하여 피부를 촉촉하게 유지해주고, 잠들어 있는 동안 피부를 진

'피부타입에 따라 추천하는 화장품은 다음과 같습니다:\n\n1. 건성 피부: Innisfree의 그린티 씨드 크림은 보습력이 뛰어나고 촉촉한 텍스처로 피부에 영양과 수분을 공급해줍니다. 녹차 추출물이 함유되어 피부를 진정시켜주고 피부 톤을 밝게 만들어줍니다.\n\n2. 수분 부족한 피부: Laneige의 워터 슬리핑 마스크는 수분을 공급하여 피부를 촉촉하게 유지해주고, 잠들어 있는 동안 피부를 진정시켜줍니다.\n\n3. 메이크업을 오랫동안 유지시키고 싶은 피부: Etude House의 플레이 컬러 아이즈는 다양한 컬러와 텍스처로 구성된 아이섀도우 팔레트로, 부드러운 발림성과 오래 지속되는 지속력을 가지고 있어 메이크업을 오랫동안 유지시켜줍니다.\n\n4. 자연스러운 커버력을 원하는 피부: Missha의 M 퍼펙트 커버 BB 크림은 커버력이 뛰어나고 자연스러운 마무리감을 가진 BB 크림입니다. 자외선 차단 기능이 있어 피부를 보호해주고, 촉촉한 텍스처로 피부에 촉촉함을 유지시켜줍니다.\n\n5. 건조한 피부: Sulwhasoo의 기초 수분 에센스는 피부에 수분을 공급하여 건조함을 개선해주고, 한방 재료를 사용하여 피부를 진정시켜주고 탄력을 증진시켜줍니다.\n\n이러한 제품들은 각각 다른 피부타입에 맞게 개발되었으며, 사용자의 피부타입에 따라 적합한 제품을 선택하여 추천해줄 수 있습니다.'

In [ ]:
# 여러개 인풋값 사용 가능한 Sequential Chain
system_template = """
사용자의 이름 {name}이 제공됩니다.
사용자의 피부타입은 {skin type}입니다.
주어진 조건으로 사용자에게 알맞은 스킨케어 방법을 알려주시오
"""

translate_prompt = PromptTemplate(
    input_variables=['name', 'skin type'],
    template=system_template
)

chain = LLMChain(llm=chat_llm,
                 prompt=translate_prompt,
                 output_key="skin care")

# 두번째 체인
second_prompt = PromptTemplate(
    input_variables=["skin care"],
    template="다음 스킨케어에 사용하면 좋은 화장품 추천 {skin care}",
)

chain_two = LLMChain(llm=chat_llm, prompt=second_prompt, output_key="cosmetic")

In [ ]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain, chain_two],
    input_variables=["name", "skin type"],
    output_variables=["skin care", "cosmetic"],
    verbose=True)
overall_chain({"name":"유진", "skin type":"DRPW"})



> Entering new SequentialChain chain...

> Finished chain.


{'name': '유진',
 'skin type': 'DRPW',
 'skin care': 'DRPW 피부타입은 다음과 같은 특징을 가지고 있습니다.\n\n- D (Dry): 건조한 피부\n- R (Redness): 피부 홍조\n- P (Pigmentation): 색소침착\n- W (Wrinkles): 주름\n\n이러한 피부타입을 고려하여 사용자에게 알맞은 스킨케어 방법은 다음과 같습니다.\n\n1. 건조한 피부를 위한 스킨케어:\n   - 보습 크림이나 로션을 사용하여 피부를 촉촉하게 유지합니다.\n   - 건조한 피부를 위한 특별한 보습 마스크를 사용할 수도 있습니다.\n   - 세안 후에는 피부를 너무 많이 문질러서는 안 되며, 부드럽게 탭탭 두드리는 방식으로 거칠지 않게 관리합니다.\n\n2. 피부 홍조를 위한 스킨케어:\n   - 피부를 진정시키기 위해 알로에 베라, 그린 티, 캘린듈라 등의 성분이 함유된 제품을 사용합니다.\n   - 피부 자극을 최소화하기 위해 부드럽게 세안하고, 뜨거운 물 대신 미온수를 사용합니다.\n   - 피부를 자극하는 화장품이나 성분을 피하고, 자외선 차단제를 사용하여 피부를 보호합니다.\n\n3. 색소침착을 위한 스킨케어:\n   - 색소침착을 개선하기 위해 비타민 C, 니아신아마이드 등의 성분이 함유된 화장품을 사용합니다.\n   - 자외선 차단제를 사용하여 피부를 보호하고, 자외선에 노출을 최소화합니다.\n   - 피부를 부드럽게 세안하고, 각질을 제거하기 위해 필요한 경우 각질제거제를 사용합니다.\n\n4. 주름을 위한 스킨케어:\n   - 주름 개선을 위해 레티놀, 펩타이드 등의 성분이 함유된 안티에이징 제품을 사용합니다.\n   - 피부를 자극하는 화장품이나 성분을 피하고, 자외선 차단제를 사용하여 피부를 보호합니다.\n   - 피부를 부드럽게 세안하고, 보습을 유지하기 위해 보습 크림이나 로션을 사용합니다.\n\n이러한 스킨케어 방법을 지속적으로 실천하면 사용자의 피부 상태를 개선시킬 수 있습니다. 하지만 개인